In [1]:
!pip install pydub
!pip install SpeechRecognition
!pip install gradio
!pip install pickle5
import tensorflow as tf
print(tf.__version__)
import sys
sys.path.append('./utils/')
from prepare_data import *
from model_helper import *
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
import time
from numpy import load
from keras.models import model_from_json
from pydub import AudioSegment
import glob
import moviepy.editor
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
import numpy as np
import pandas as pd
import nltk
import string
import pickle5 as pickle
import re
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gradio as gr
from pandas import DataFrame

2.4.1


In [2]:
import os
import cv2
import time
import numpy as np
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.applications.resnet50 import preprocess_input
from keras.applications import *
from keras import regularizers
import cv2
import math
import os
import time

import os
import glob
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
tf.compat.v1.disable_eager_execution()

In [4]:
class ABLSTM(object):
    def __init__(self, config):
        self.max_len = config["max_len"]
        self.hidden_size = config["hidden_size"]
        self.vocab_size = config["vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.n_class = config["n_class"]
        self.learning_rate = config["learning_rate"]

        # placeholder
        self.x = tf.compat.v1.placeholder(tf.int32, [None, self.max_len])
        self.label = tf.compat.v1.placeholder(tf.int32, [None])
        self.keep_prob = tf.compat.v1.placeholder(tf.float32)

    def build_graph(self):
        print("building graph")
        # Word embedding
        embeddings_var = tf.Variable(tf.random.uniform([self.vocab_size, self.embedding_size], -1.0, 1.0),
                                     trainable=True)
        batch_embedded = tf.nn.embedding_lookup(params=embeddings_var, ids=self.x)

        rnn_outputs, _ = bi_rnn(tf.compat.v1.nn.rnn_cell.LSTMCell(self.hidden_size),
                                tf.compat.v1.nn.rnn_cell.LSTMCell(self.hidden_size),
                                inputs=batch_embedded, dtype=tf.float32)

        fw_outputs, bw_outputs = rnn_outputs

        W = tf.Variable(tf.random.normal([self.hidden_size], stddev=0.1))
        H = fw_outputs + bw_outputs  # (batch_size, seq_len, HIDDEN_SIZE)
        M = tf.tanh(H)  # M = tanh(H)  (batch_size, seq_len, HIDDEN_SIZE)

        self.alpha = tf.nn.softmax(tf.reshape(tf.matmul(tf.reshape(M, [-1, self.hidden_size]),
                                                        tf.reshape(W, [-1, 1])),
                                              (-1, self.max_len)))  # batch_size x seq_len
        r = tf.matmul(tf.transpose(a=H, perm=[0, 2, 1]),
                      tf.reshape(self.alpha, [-1, self.max_len, 1]))
        r = tf.squeeze(r)
        h_star = tf.tanh(r)  # (batch , HIDDEN_SIZE

        h_drop = tf.nn.dropout(h_star, rate=1 - (self.keep_prob))

        # Fully connected layer（dense layer)
        FC_W = tf.Variable(tf.random.truncated_normal([self.hidden_size, self.n_class], stddev=0.1))
        FC_b = tf.Variable(tf.constant(0., shape=[self.n_class]))
        y_hat = tf.compat.v1.nn.xw_plus_b(h_drop, FC_W, FC_b)

        self.loss = tf.reduce_mean(
            input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=self.label))

        # prediction
        self.prediction = tf.argmax(input=tf.nn.softmax(y_hat), axis=1)

        # optimization
        loss_to_minimize = self.loss
        tvars = tf.compat.v1.trainable_variables()
        gradients = tf.gradients(ys=loss_to_minimize, xs=tvars, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
        grads, global_norm = tf.clip_by_global_norm(gradients, 1.0)

        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.train_op = self.optimizer.apply_gradients(zip(grads, tvars), global_step=self.global_step,
                                                       name='train_step')
        print("graph built successfully!")

In [5]:
config = {
        "max_len": 100,
        "hidden_size": 64,
        "vocab_size": 161177,
        "embedding_size": 128,
        "n_class": 2,
        "learning_rate": 1e-3,
        "batch_size": 4,
        "train_epoch": 5
    }

In [6]:
classifier = ABLSTM(config)
classifier.build_graph()

building graph
graph built successfully!


In [7]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [8]:
tf.compat.v1.train.Saver().restore(sess, tf.compat.v1.train.latest_checkpoint('./attn_bilstm/'))

In [9]:
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    aud1 = []
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
                #aud1.append(whole_text)
    aud1.append(whole_text)           
    # return the text for all chunks detected
    return aud1

In [10]:
r = sr.Recognizer()
lem = WordNetLemmatizer()
with open('./data/tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/mridul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mridul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mridul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def audio_prediction(video):
  video=moviepy.editor.VideoFileClip(video)
  audio=video.audio
  audio.write_audiofile('./sample_video/audio/audio.wav')

  audio_path = './sample_video/audio/audio.wav'


  aud1 = get_large_audio_transcription(audio_path)

  aud1_str = aud1[0]
  sentences = aud1_str.split('.')
  all_sentences = []
  for sentence in sentences:
    sentence_new = sentence.lower()
    sentence_new = sentence_new.strip()
    all_sentences.append(sentence_new)

  all_sentences1 = []
  for sentence in all_sentences:
    if len(sentence) > 1:
      all_sentences1.append(sentence)

  length = []
  total_words = 0
  for tweet in all_sentences1:
    len_tweet = len(tweet.split(' '))
    total_words += len_tweet
    length.append(len_tweet)

  video_duration = int(video.duration)
  factor = 0
  if total_words > 0:
    factor = video_duration/total_words

  starting_time = []
  ending_time = []
  ending_val = 0
  starting_time.append(0)
  for l in length:
    ending_val = ending_val + l*factor
    starting_time.append(ending_val)
    ending_time.append(ending_val)
  starting_time = starting_time[0:-1]

  time_stamps = []
  for i in range(len(starting_time)):
    start = starting_time[i]
    end = ending_time[i]
    h1 = int(start // 3600)
    m1 = int(start // 60)
    s1 = int(start - h1*3600 - m1 * 60)
    h2 = int(end // 3600)
    m2 = int(end // 60)
    s2 = int(end - h2*3600 - m2 * 60)
    generated_time_stamp = str(h1)+':'+str(m1)+':'+str(s1)+'-'+str(h2)+':'+str(m2)+':'+str(s2)
    time_stamps.append(generated_time_stamp)

  all_sentences_final = []
  for sentence in all_sentences1:
    new_sentence = ""
    all_words = word_tokenize(sentence)
    for word in all_words:
      if word not in stop_words:
        new_word = lem.lemmatize(word)
        new_sentence = new_sentence + ' ' + word
    new_sentence = new_sentence[1:]
    all_sentences_final.append(new_sentence)

  map = dict(zip(all_sentences_final, range(len(all_sentences_final))))

  preprocessed_sentences = []
  for sentence in all_sentences_final:
    if len(sentence) > 1:
      preprocessed_sentences.append(sentence)

  df = DataFrame(preprocessed_sentences, columns=['tweet'])
  df.dropna(axis=0, inplace=True)
  x_tf=df['tweet']

  x_tf.dropna(axis=0,inplace=True)
  prediction_data = pad_sequences(tokenizer.texts_to_sequences(x_tf), maxlen=100)
  prediction_output = np.zeros(len(prediction_data))
  feed_dict = make_test_feed_dict(classifier, (prediction_data, prediction_output))
  prediction = sess.run(classifier.prediction, feed_dict)
  percentage_hate = np.count_nonzero(prediction == 1)

  result = 'The hate sentences are as follows:\n'
  for i in range(len(prediction)):
    if prediction[i] == 1:
      index = map[preprocessed_sentences[i]]
      result = result + all_sentences1[index] +' (' + time_stamps[index] + ')\n'

  if percentage_hate > 0.04:
    return 'Violent', result
  else:
    return 'Non-Violent', 'No time stamps as video is non violent'

In [12]:
def video_to_frames(input_loc, output_loc,video_name):

    count = 0
    cap = cv2.VideoCapture(input_loc)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    print (frameRate)

    while(cap.isOpened()):
        frameId = cap.get(10) #current frame number
        # print(frameId)
        ret, frame = cap.read()
        
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            frame = cv2.resize(frame, (224,224))
            filename = video_name + "frame%d.jpg" % count;count+=1
            cv2.imwrite(output_loc + filename, frame)
            
    cap.release()
    print ("Done!")




In [13]:
def create_seq_single_video(frames):
    
    print('+++ Creating Sequence... +++')
    vid = []
#     frames = np.array(frames)
    i = 0
    while i < len(frames):
        vid.append(frames[i:i+30])
        i = i+30
    vid = np.asarray(vid)
    
    return vid
    



In [14]:

def preprocess_lstm(features, frames):
    frames = np.asarray(frames)
    v_features = features[0:30*math.floor(frames.shape[0]/30)]
    
    print("Video Features Shape: ", v_features.shape)
    
    vid = create_seq_single_video(v_features)
    
    test_x = vid
    print("Total data: ", test_x.shape)
    test_x = np.reshape(test_x, (test_x.shape[0],test_x.shape[1],np.prod(test_x.shape[2:])))
    print("(LSTM) After Rehshape: ", test_x.shape)
    
    return test_x




In [15]:
def feature_extract(frames, model):
    
    print("+++ Extracting feature... +++")
    
    frames = np.asarray(frames)
    
    print("Before Feaure extraction: ")
    print(frames.shape)
    
    all_data = frames
    print("Adding all data: ", all_data.shape)
    
    #creates feature descriptors
    all_data = all_data.astype('float64')
    desc = preprocess_input(all_data)
    if(model == 'resnet50'):
        loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg19'):
        loaded_model = VGG19(input_shape=(224,224,3), include_top=False)
    elif(model == 'vgg16'):
        loaded_model = VGG16(input_shape=(224,224,3), include_top=False)
    else:
        print("Please give model name - 'resnet50', 'vgg19', 'vgg16'")
        
    loaded_model = Model(loaded_model.input,loaded_model.output)
    features = loaded_model.predict(desc,batch_size=10,verbose=1)

    print ("After Feature extraction: ", features.shape)
    
    return features

In [16]:
frames_output_path = './sample_video/frames_output/'
v_path = r"./sample_video/frames_output/"

In [17]:
def video_prediction(video):
  video_file = moviepy.editor.VideoFileClip(video)
  video_duration = int(video_file.duration)
  print("+++ Removing all the frames from the source folder... +++")
  files = glob.glob(r"./sample_video/frames_output/*")
  for f in files:
      os.remove(f)

  video_to_frames(video,frames_output_path,'video')
  out_frames = []

  for frame in os.listdir(v_path):
      frame = cv2.imread(os.path.join(v_path,frame))
      out_frames.append(frame)


  print("++++++++++ Resnet50 Features Extraction ++++++++")
  features = feature_extract(out_frames, 'resnet50')

  print("++++++++++ Preprocesing LSTM +++++++++++++++++++")
  test_x = preprocess_lstm(features, out_frames)

  model = Sequential()
  model.add(LSTM(50, input_shape=(test_x.shape[1],test_x.shape[2]), return_sequences=False, kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dense(1,activation='sigmoid'))
  #model.load_weights('resnet_LSTM_v1.h5')
  model.load_weights('./videoModel/resnet_LSTM_wwe.h5')
  model.summary()
  optimizer = optimizers.Adam(lr=0.001,decay=0.004)
  model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

  print("+++ Generating result... +++")
      
  pred = model.predict(test_x)

  final_out = [1 if x > 0.5 else 0 for x in pred ]


  factor = video_duration/len(final_out)

  time_stamps = []
  start = end = 0
  flag = False
  for i in range(len(final_out)):
    if final_out[i] == 1 and flag == False:
      start = i*factor
      flag = True
    elif final_out[i] == 1 and flag == True:
      continue
    elif final_out[i] == 0 and flag == True:
      end = i*factor
      flag = False
      time_stamps.append((start, end))
    else:
      continue
  if final_out[-1] == 1 and flag == True:
    time_stamps.append((start, video_duration))

  result = 'The hate time stamps are as follows:\n'

  for (start, end) in time_stamps:
    h1 = int(start // 3600)
    m1 = int(start // 60)
    s1 = int(start - h1*3600 - m1 * 60)
    h2 = int(end // 3600)
    m2 = int(end // 60)
    s2 = int(end - h2*3600 - m2 * 60)
    time_stamp = str(h1)+':'+str(m1)+':'+str(s1)+'-'+str(h2)+':'+str(m2)+':'+str(s2)+'\n'
    result = result + time_stamp

  print("+++ Final Output... +++")
  print(pd.DataFrame(final_out)[0].value_counts())


  df = pd.DataFrame(final_out)
  video_result = ''
  try:
    string1 = "Percentage of non-violent sequence of frames : {0}".format(list(df.value_counts(normalize=True)[1].values*100)[0])
    string2 = "Percentage of violent sequence of frames : {0}".format(list(df.value_counts(normalize=True)[0].values*100)[0])
    print(string1)
    print(string2)
    video_result = string1 + '\n' + string2
  except:
    pass

  try:
    if list(df.value_counts(normalize=True)[1].values*100)[0] > 80:
      return "NON-VIOLENT", result
    else:
      return "VIOLENT", result
  except:
    return "VIOLENT", 'No time stamps as video is not violent'

In [18]:
iface = gr.Interface([audio_prediction, video_prediction], inputs="video", outputs=['text', 'text'], title="Violence Detection")

In [ ]:
iface.launch(debug=True)

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


chunk:   0%|          | 0/1242 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./sample_video/audio/audio.wav


MoviePy - Done.
+++ Removing all the frames from the source folder... +++
25.0
Done!
++++++++++ Resnet50 Features Extraction ++++++++
+++ Extracting feature... +++
Before Feaure extraction: 
(1405, 224, 224, 3)
Adding all data:  (1405, 224, 224, 3)
After Feature extraction:  (1405, 7, 7, 2048)
++++++++++ Preprocesing LSTM +++++++++++++++++++
Video Features Shape:  (1380, 7, 7, 2048)
+++ Creating Sequence... +++
Total data:  (46, 30, 7, 7, 2048)
(LSTM) After Rehshape:  (46, 30, 100352)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                20080600  
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 20,080,651
Trainable params: 20,080,651
Non-trainable params: 0
_________________________________________________________________
+++ Generating r

chunk:   0%|          | 0/1165 [00:00<?, ?it/s, now=None]

...

MoviePy - Writing audio in ./sample_video/audio/audio.wav


MoviePy - Done.


audio-chunks/chunk1.wav : Hair. 
Error: 
Error: 


[2021-06-04 05:44:12,052] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/home/mridul/anaconda3/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1375, in _do_call
    return fn(*args)
  File "/home/mridul/anaconda3/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1359, in _run_fn
    return self._call_tf_sessionrun(options, feed_dict, fetch_list,
  File "/home/mridul/anaconda3/lib/python3.8/site-packages/tensorflow/python/client/session.py", line 1451, in _call_tf_sessionrun
    return tf_session.TF_SessionRun_wrapper(self._session, options, feed_dict,
tensorflow.python.framework.errors_impl.InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: In[0] ndims must be >= 2: 1
	 [[{{node xw_plus_b/MatMul}}]]
  (1) Invalid argument: In[0] ndims must be >= 2: 1
	 [[{{node xw_plus_b/MatMul}}]]
	 [[ArgMax/_77]]
0 successful operations.
0 derived errors ignored.

During handling of the above ex

Error: 
